In [ ]:
## mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing in

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import gradio as gr

# Load the saved model
model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/network_traffic_classifier.keras')

# Define the attack dictionary
attack_dict = {
    'normal': 'normal',
    'DoS': 'DoS',
    'Probe': 'Probe',
    'R2L': 'R2L',
    'U2R': 'U2R'
}

# Define the preprocessing function
def preprocess_data(df):
    required_columns = ['protocol_type', 'service', 'flag'] + [f'feature_{i}' for i in range(1, 34)]  # Assuming 36 features

    # Ensure the dataframe has the correct columns
    if len(df.columns) != len(required_columns):
        raise ValueError(f"Expected {len(required_columns)} columns, but got {len(df.columns)}.")

    # Convert categorical columns to numeric using Label Encoding
    df['protocol_type'] = df['protocol_type'].astype('category').cat.codes
    df['service'] = df['service'].astype('category').cat.codes
    df['flag'] = df['flag'].astype('category').cat.codes

    # Standardize the feature columns
    df.iloc[:, :-1] = (df.iloc[:, :-1] - df.iloc[:, :-1].mean()) / df.iloc[:, :-1].std()

    return df

# Function to map predictions to attack classes
def map_predictions(predictions):
    # Ensure the number of unique classes in predictions matches the attack_dict
    inverse_attack_dict = {i: v for i, v in enumerate(attack_dict.values())}

    # Fallback for classes not found in dictionary
    mapped_preds = [inverse_attack_dict.get(pred, "Unknown") for pred in predictions]

    return mapped_preds

# Function to make predictions from the uploaded file
def predict_from_file(file):
    # Read the uploaded CSV file
    df = pd.read_csv(file.name)  # Access the file content with .name

    # Preprocess the data
    df_preprocessed = preprocess_data(df)

    # Make predictions
    predictions = model.predict(df_preprocessed)
    predicted_classes = np.argmax(predictions, axis=1)

    # Debugging: print predicted classes and ensure they match the attack dictionary
    print("Predicted classes:", predicted_classes)

    # Map predictions back to attack categories
    final_predictions = map_predictions(predicted_classes)

    # Return predictions as a single string
    return ', '.join(final_predictions)

# Create the Gradio interface
iface = gr.Interface(
    fn=predict_from_file,
    inputs=gr.File(label="Upload CSV File"),  # Use gr.File instead of gr.inputs.File
    outputs=gr.Textbox(label="Predictions"),   # Use gr.Textbox to handle multiple predictions
    live=False
)

# Launch the interface
iface.launch(debug=True)


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://c49cee3187dd6f9d50.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Predicted classes: [0 0 0 0 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted classes: [0 0 0 0 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Predicted classes: [0 0 0 0 0]


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 321, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1935, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1520, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 8

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predicted classes: [0 0 0 0 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Predicted classes: [0 0 0 0 0]


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 321, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1935, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1520, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 8

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predicted classes: [0 0 0 0 0]
